In [3]:
import psutil
import time
import os
from IPython.display import clear_output

clear_output(wait=False)

current_pid = os.getpid()
print(f'setting high prio for set_caiman_nice self pid = {current_pid}')
psutil.Process(os.getpid()).nice(psutil.ABOVE_NORMAL_PRIORITY_CLASS)

local_py_pids = []
other_user_pids = []
# while True:
while psutil.cpu_percent() > 10:
    print(f'cpu util = {psutil.cpu_percent()}, start setting low prio for current user\'s caiman process.')

    for proc in psutil.process_iter(): # ['pid', 'name', 'username']
        try:
            if ("python" in proc.name()) and (os.getlogin() in proc.username() and (proc.pid not in other_user_pids)):
                local_py_pids.append(proc.pid)
        except psutil.AccessDenied:
            other_user_pids.append(proc.pid)
            pass

    local_py_pids.remove(os.getpid()) # get pid for all current user's python processes except for this script itself
    for pid in local_py_pids:
        if psutil.Process(pid).nice() != psutil.BELOW_NORMAL_PRIORITY_CLASS:
            psutil.Process(pid).nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
            print("lower prio of process {} owned by {}".format(pid, psutil.Process(pid).username()))


    time.sleep(60*30) # check for new python process every n sec
    clear_output(wait=True) # clear cell output after a pause, otherwise too lengthy


setting high prio for set_caiman_nice self pid = 89316


In [43]:
%%timeit
psutil.Process(pid).nice(psutil.NORMAL_PRIORITY_CLASS)
# psutil.Process(pid).nice()


120 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [44]:
%%timeit
# psutil.Process(pid).nice(psutil.NORMAL_PRIORITY_CLASS)
psutil.Process(pid).nice()


13.7 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [33]:
%%timeit

cpu_util = psutil.cpu_percent()
print(f'cpu util = {cpu_util}, start setting caiman nice')

local_py_pids = []
other_user_pids = []

# while psutil.cpu_percent() > 10:
# while True:
for proc in psutil.process_iter(): # ['pid', 'name', 'username']
    try:
        if ("python" in proc.name()) and (os.getlogin() in proc.username() and (proc.pid not in other_user_pids)):
            local_py_pids.append(proc.pid)
    except psutil.AccessDenied:
        other_user_pids.append(proc.pid)
        pass

print(f'setting low prio for current user\'s caiman process.')
local_py_pids.remove(os.getpid()) # get pid for all python processes except for self
for pid in local_py_pids:
    if psutil.Process(pid).nice() != psutil.BELOW_NORMAL_PRIORITY_CLASS:
        psutil.Process(pid).nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
        print("lower prio of process {} owned by {}".format(pid, psutil.Process(pid).username()))

cpu util = 100.0, start setting caiman nice
setting low prio for current user's caiman process.
cpu util = 100.0, start setting caiman nice
setting low prio for current user's caiman process.
cpu util = 100.0, start setting caiman nice
setting low prio for current user's caiman process.
cpu util = 100.0, start setting caiman nice
setting low prio for current user's caiman process.
cpu util = 100.0, start setting caiman nice
setting low prio for current user's caiman process.
cpu util = 100.0, start setting caiman nice
setting low prio for current user's caiman process.
cpu util = 100.0, start setting caiman nice
setting low prio for current user's caiman process.
cpu util = 100.0, start setting caiman nice
setting low prio for current user's caiman process.
16.8 s ± 860 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
%%timeit

cpu_util = psutil.cpu_percent()
print(f'cpu util = {cpu_util}, setting low prio for current user\'s caiman process.')

# local_py_pids = []
other_user_pids = []

# while psutil.cpu_percent() > 10:
# while True:
for proc in psutil.process_iter(): # ['pid', 'name', 'username']
    try:
        if ("python" in proc.name()) and (os.getlogin() in proc.username() and (proc.pid not in other_user_pids) and (proc.pid != os.getpid())):
            pid = proc.pid
            if psutil.Process(pid).nice() != psutil.BELOW_NORMAL_PRIORITY_CLASS:
                psutil.Process(pid).nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
                print("lower prio of process {} owned by {}".format(pid, psutil.Process(pid).username()))
    except psutil.AccessDenied:
        other_user_pids.append(proc.pid)


cpu util = 100.0, setting low prio for current user's caiman process.
cpu util = 100.0, setting low prio for current user's caiman process.
cpu util = 100.0, setting low prio for current user's caiman process.
cpu util = 100.0, setting low prio for current user's caiman process.
cpu util = 100.0, setting low prio for current user's caiman process.
cpu util = 100.0, setting low prio for current user's caiman process.
cpu util = 100.0, setting low prio for current user's caiman process.
cpu util = 100.0, setting low prio for current user's caiman process.
17.6 s ± 901 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit

import pandas as pd

cpu_util = psutil.cpu_percent()
print(f'cpu util = {cpu_util}, start setting caiman nice')

df = pd.DataFrame(columns=['pid', 'name', 'username'])
for proc in psutil.process_iter(['pid', 'name', 'username']):
    if ("python" in proc.name()):
        df = df.append(proc.info, ignore_index=True)

# local_py_pids = []
# other_user_pids = []

# # while psutil.cpu_percent() > 10:
# # while True:
# for proc in psutil.process_iter(): # ['pid', 'name', 'username']
#     try:
#         if ("python" in proc.name()) and (os.getlogin() in proc.username() and (proc.pid not in other_user_pids)):
#             local_py_pids.append(proc.pid)
#     except psutil.AccessDenied:
#         other_user_pids.append(proc.pid)
#         pass

# print(f'setting low prio for current user\'s caiman process.')
# local_py_pids.remove(os.getpid()) # get pid for all python processes except for self
# for pid in local_py_pids:
#     if psutil.Process(pid).nice() != psutil.BELOW_NORMAL_PRIORITY_CLASS:
#         psutil.Process(pid).nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
#         print("lower prio of process {} owned by {}".format(pid, psutil.Process(pid).username()))

cpu util = 100.0, start setting caiman nice
cpu util = 100.0, start setting caiman nice
cpu util = 100.0, start setting caiman nice
cpu util = 100.0, start setting caiman nice
cpu util = 100.0, start setting caiman nice
cpu util = 100.0, start setting caiman nice
cpu util = 100.0, start setting caiman nice
cpu util = 100.0, start setting caiman nice
17.7 s ± 806 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
import pandas as pd

cpu_util = psutil.cpu_percent()
print(f'cpu util = {cpu_util}, start setting caiman nice')

df = pd.DataFrame(columns=['pid', 'name', 'username'])
for proc in psutil.process_iter(['pid', 'name', 'username']):
    if ("python" in proc.name()):
        df = df.append(proc.info, ignore_index=True)
df

cpu util = 100.0, start setting caiman nice


,pid,name,username
0,11364,python.exe,None
1,14144,python.exe,None
2,15640,python.exe,None
3,21156,python.exe,None
4,24460,python.exe,None
...,...,...,...
69,89316,python.exe,NB-HUBEL\GlickfeldLab
70,89452,python.exe,None
71,89456,python.exe,None
72,89504,python.exe,None
